In [37]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt

In [38]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [39]:
img= cv2.imread("imgcropped.jpg",0) 
print(np.amax(img))
cv2.imwrite("test.jpg",img)
if(np.amax(img)==1):
    img=img.astype("uint8")
else:
    img=(img/255).astype("uint8")
img=1-img
print(img)
proj = np.sum(img,1)
#proj=1-proj
maxProjection = np.max(proj) 
w = 500 
result = np.zeros(img.shape) 
width=img.shape[1] 
binOrig=np.copy(img) 
# Draw a line for each row 
for row in range(img.shape[0]):
    if(proj[row]>.8*maxProjection): 
        img[row]=1 
        cv2.line(result, (0,row), (int(proj[row]*(maxProjection/width)),row), (255,255,255), 1) 
cv2.imwrite('result.png', result) 
result=1-result
print(proj.shape, img.shape)

255
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(315,) (315, 945)


In [40]:
def calcStaffPos(rowHist):
    n,m=rowHist.shape
    thres=int((maxProjection)/2)
    print(thres,m)
    start=end=0
    print(rowHist)
    width=[]
    #rowHist=(rowHist/255).astype("uint8")
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                #print(start,end)
                peaksMid.append((start+end)/2)
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid).astype("uint8")

width,peaksMids=calcStaffPos(result)
print(width,peaksMids)

302 945
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[2 2 5 2 2 2 2 2 2 6 6 6 2 2 2] [ 50  52  59  67  69  72  74  76  84 191 199 207 214 216 223]


In [41]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum>thresPixel):
            print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=1
    return imgOriginal


img= cv2.imread("imgcropped.jpg",0) 
img=img/255
img=img.astype('uint8')
imgCopy=np.copy(img)
cv2.imwrite("imgCopy.png",img*255)

for i in range(len(peaksMids)):
    imgCopy=removeStaffRow(imgCopy,peaksMids[i],width[i])

cv2.imwrite("outing.png",imgCopy*255)
    

 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1

True